In [2]:
#set up autoreload
%load_ext autoreload
%autoreload 2

import polars as pl
from pathlib import Path

import importlib  
fa_etl = importlib.import_module("fa-etl")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
county = "36061"
path_to_dir = "/Users/claireboyd/internships/mansueto/firstamerican-etl/"
output_filepath_ranked = path_to_dir+f"dev/{county}/staging/ranked_ValHist{county}.parquet"

taxhist = pl.read_parquet(path_to_dir+f"dev/{county}/staging/TaxHist{county}.parquet")
valhist = pl.read_parquet(path_to_dir+f"dev/{county}/staging/ValHist{county}.parquet")
annual = pl.read_parquet(path_to_dir+f"dev/{county}/staging/Prop{county}.parquet")
sales_ranked = pl.read_parquet(path_to_dir+f"dev/{county}/staging/ranked_Deed{county}.parquet")
sales = pl.read_parquet(path_to_dir+f"dev/{county}/staging/Deed{county}.parquet")

appr = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/appr.parquet", low_memory=True)
assd = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/assd.parquet", low_memory=True)
market = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/market.parquet", low_memory=True)

In [70]:
assd.join(
    other=market,
    how="left",
    on=['PropertyID', 'Year'],
).sink_parquet(Path(output_filepath_ranked), compression="snappy")


In [71]:
valhist = pl.read_parquet(path_to_dir+f"dev/{county}/staging/ranked_ValHist{county}.parquet")

In [72]:
valhist.shape

(3566623, 4)

In [43]:
lf = pl.LazyFrame(
    {
        "a": [1, 2, 3, 4],
        "b": [0.5, 4, 10, 13],
        "c": [True, True, False, True],
    }
)

In [56]:
lf.with_columns([
    pl.when(True).then(None).alias("ApprTotalValue")
]).collect()

a,b,c,Appr
i64,f64,bool,null
1,0.5,true,null
2,4.0,true,null
3,10.0,false,null
4,13.0,true,null


In [12]:
sales.filter(
    pl.col("PropIDFlag") == 1
)

PropertyID,SaleAmt,RecordingDate,FIPS,FATimeStamp,FATransactionID,TransactionType,SaleDate,RecordingYearSlice,PropertyID_str,FATransactionID_1,RecordingYear,SaleYear,FATimeStampYear,SaleFlag,PropIDFlag
i64,i64,date,str,date,i64,str,date,str,str,str,i64,i32,i32,i32,i32


In [12]:
merged.filter([
    pl.col("SaleAmt").is_not_null(),
    pl.col("Year") > 2017
]).select(
    ['PropertyID','Year','Value','AssessmentUsed','SaleAmt', 'TaxAmt', 'TaxAmtAdjusted']
).with_columns([
    (pl.col('TaxAmt')/pl.col('Value')).alias("ApproxTaxRate")
])

PropertyID,Year,Value,AssessmentUsed,SaleAmt,TaxAmt,TaxAmtAdjusted,ApproxTaxRate
i64,i64,i64,str,i64,i64,f64,f64
91847870,2018,117677,"""Assd""",1580000,1310424,13104.24,11.13577
91849008,2018,247162,"""Assd""",4400000,3114052,31140.52,12.599235
91848046,2018,71155,"""Assd""",712775,773352,7733.52,10.868555
91846549,2018,149310,"""Assd""",2475000,1527372,15273.72,10.229536
91847537,2018,128574,"""Assd""",1625000,1496148,14961.48,11.636474
91848333,2018,356895,"""Assd""",7400000,3958776,39587.76,11.092271
91846649,2018,219589,"""Assd""",3100000,2518932,25189.32,11.471121
91847215,2018,299300,"""Assd""",2825000,3327036,33270.36,11.116057
91846134,2022,718067,"""Assd""",12500000,8808528,88085.28,12.267
